In [ ]:
# https://adventofcode.com/2018/day/24

import Base.parse

mutable struct Group
    race::Symbol
    units::Int
    hp::Int
    ap::Int
    ip::Int
    attack::Symbol
    immunity::Set{Symbol}
    weakness::Set{Symbol}
end

function parse(::Type{T}, race::String, line::String) where T<:Group
    regex = r"(\d+) units each with (\d+) hit points \(?(.*?)\)? ?with an attack that does (\d+) (\w+) damage at initiative (\d+)"
    units, hp, immuweak, ap, attack, ip = match(regex, line).captures
    
    immunity = if occursin("immune", immuweak)
        split(match(r"immune to ([^;]+)", immuweak).captures[1], ", ")
        else [] end

    weakness = if occursin("weak", immuweak)
        split(match(r"weak to ([^;]+)", immuweak).captures[1], ", ")
        else [] end
    
    Group(Symbol(replace(race, r"[ :]"=>"")),
          parse(Int, units),
          parse(Int, hp),
          parse(Int, ap),
          parse(Int, ip),
          Symbol(attack),
          Set(Symbol.(immunity)),
          Set(Symbol.(weakness)))
end

data = open("24.txt", "r") do file
    groups = []
    for _ in 1:2
        race = readline(file)
        while true
            line = readline(file)
            if line == "" break end
            push!(groups, parse(Group, race, line))
        end
    end
    groups
end


In [21]:
# part 1

power(g) = g.units * g.ap

damage(a, d) = 
    if a.attack in d.immunity
        0
    elseif a.attack in d.weakness
        2 * power(a)
    else
        power(a)
    end

function simulate(groups)
    groups = deepcopy(groups)
    while length(unique(g.race for g in groups)) > 1
        # planning
        sort!(groups, by=g -> (power(g), g.ip), rev=true)
        targets = Dict()
        for a in groups
            d = sort([d for d in groups
                      if d.race != a.race
                         && damage(a, d) != 0
                         && d ∉ values(targets)],
                     by=d->(damage(a, d), power(d), d.ip),
                     rev=true)
            if !isempty(d)
                targets[a] = d[1]
            end
        end

        # attack
        sort!(groups, by=g->g.ip, rev=true)
        stalemate = true
        for a in groups
            if haskey(targets, a)
                d = targets[a]
                k = div(damage(a, d), d.hp)
                if a.units > 0 && k > 0
                    d.units -= k
                    stalemate = false
                end
            end
        end

        # resolution
        if stalemate break end
        filter!(g->g.units > 0, groups)
    end

    return groups
end
                
sum(g.units for g in simulate(data))


20340

In [24]:
# part 2

function simulate(groups, boost)
    groups = deepcopy(groups)
    for g in groups
        if g.race == :ImmuneSystem g.ap += boost end
    end
    simulate(groups)
end

# bisect to find high end successful boost
boost = 1
while true
    groups = simulate(data, boost)
    if unique(g.race for g in groups) == [:ImmuneSystem] break end
    boost *= 2
end

# linear search to find low end successful boost
boost = div(boost, 2)
while true
    boost += 1
    groups = simulate(data, boost)
    if unique(g.race for g in groups) == [:ImmuneSystem] break end
end

sum(g.units for g in simulate(data, boost))


3862